In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm
import re


In [2]:
def getSoup(link):
    sleep(0.05)
    r = requests.get(link)
    return BeautifulSoup(r.text,'lxml')

In [3]:
def getAllLinks(link):
    soup = getSoup(link)
    table = soup.find('table', {'id':'reportable'})
    tds = table.findAll('td', {'align':'left'})
    a = [td.find('a') for td in tds]
    res = [i for i in a if i]
    return res


In [4]:
links = getAllLinks('http://tennisabstract.com/reports/atpRankings.html')


In [5]:
b = [str(links[i])for i in range(len(links))]
c = [i.replace('"', '!') for i in b]
d = [o.split("!") for o in c]
flinks = [d[p][1] for p in range(len(d))]

In [6]:
Namesss = [flink.replace('http://www.tennisabstract.com/cgi-bin/player.cgi?p=', '') for flink in flinks]
Names=Namesss[:200]

In [7]:
finallist = []
ranx = 0
for nam in Names:
    ranx += 1
    soop = getSoup('http://www.minorleaguesplits.com/tennisabstract/cgi-bin/frags/'+nam+'.js')
    playerlist = []
    name_str = nam
    name_list = []
    name_list = re.findall('[A-Z][^A-Z]*', name_str)
    namstr = ' '.join(map(str, name_list))
    playerlist.append(namstr)
    playerlist.append(ranx)
    if (soop.find('table', {'id':'tour-years'})):
        statstable = soop.find('table', {'id':'tour-years'})
        statsdfs = pd.read_html(str(statstable), index_col = 0)
        statsdf = statsdfs[0]
        playerlist.append(statsdf.at['Career', 'Hld%'])
        playerlist.append(statsdf.at['Career', 'Brk%'])
    else:
        playerlist.append("No data")
        playerlist.append("No data")
    finallist.append(playerlist)

In [8]:
finalldf = pd.DataFrame(finallist, columns = ['Name', 'Rank', 'Hld%', 'Brk%'])
finalldf

,Name,Rank,Hld%,Brk%
0,Novak Djokovic,1,85.8%,32.1%
1,Rafael Nadal,2,85.9%,33.4%
2,Dominic Thiem,3,83.2%,23.7%
3,Daniil Medvedev,4,82.2%,25.2%
4,Roger Federer,5,88.8%,26.7%
...,...,...,...,...
195,Aleksandar Vukic,196,73.3%,6.9%
196,Robin Haase,197,78.0%,20.5%
197,Ernesto Escobedo,198,80.2%,12.5%
198,Lukas Rosol,199,78.8%,16.8%


In [9]:
for i in range(len(Names)):    
    finalldf.rename(columns={},index={i:finalldf.at[i,"Name"]},inplace=True)
   
print(finalldf)

                              Name  Rank   Hld%   Brk%
Novak Djokovic      Novak Djokovic     1  85.8%  32.1%
Rafael Nadal          Rafael Nadal     2  85.9%  33.4%
Dominic Thiem        Dominic Thiem     3  83.2%  23.7%
Daniil Medvedev    Daniil Medvedev     4  82.2%  25.2%
Roger Federer        Roger Federer     5  88.8%  26.7%
...                            ...   ...    ...    ...
Aleksandar Vukic  Aleksandar Vukic   196  73.3%   6.9%
Robin Haase            Robin Haase   197  78.0%  20.5%
Ernesto Escobedo  Ernesto Escobedo   198  80.2%  12.5%
Lukas Rosol            Lukas Rosol   199  78.8%  16.8%
Lukas Lacko            Lukas Lacko   200  76.4%  16.5%

[200 rows x 4 columns]


In [10]:
class Tennisplayer:
    def __init__(self,name):
        self.name = name
        
    def ranking(self,name):
        return finalldf.at[name,"Rank"]
    def servis(self,name):
        return finalldf.at[name,"Hld%"]
    def returny(self,name):
        return finalldf.at[name,"Brk%"]

In [11]:
povrchy=["Clay","Hard","Grass"]
#tři vstupy, výstupy pravděpodobnost výhry servisů(returnu)
import tkinter
from tkinter import *
from tkinter import messagebox
def proces():
    Tennisplayer1=Entry.get(E1)
    Tennisplayer2=Entry.get(E2)
    Surface=Entry.get(E3)
    h1=Tennisplayer(Tennisplayer1)
    h2=Tennisplayer(Tennisplayer2)
    Tennisplayer1ranking=h1.ranking(Tennisplayer1)
    Tennisplayer2ranking=h2.ranking(Tennisplayer2)
    Tennisplayer1servis=h1.servis(Tennisplayer1)
    if Tennisplayer1servis=="No data":
        Entry.insert(E4,0,"No data")
        print("No data")
        Entry.insert(E5,0,"No data")
        print("No data")
        return None
    Tennisplayer2servis=h2.servis(Tennisplayer2)
    if Tennisplayer2servis=="No data":
        Entry.insert(E4,0,"No data")
        print("No data")
        Entry.insert(E5,0,"No data")
        print("No data")
        return None       
    Tennisplayer1returny=h1.returny(Tennisplayer1)
    if Tennisplayer1returny=="No data":
        Entry.insert(E4,0,"No data")
        print("No data")
        Entry.insert(E5,0,"No data")
        print("No data")
        return None    
    Tennisplayer2returny=h2.returny(Tennisplayer2)
    if Tennisplayer2returny=="No data":
        Entry.insert(E4,0,"No data")
        print("No data")
        Entry.insert(E5,0,"No data")
        print("No data")
        return None     
    Tennisplayer1servis=Tennisplayer1servis[:-1]
    Tennisplayer2servis=Tennisplayer2servis[:-1]
    Tennisplayer1returny=Tennisplayer1returny[:-1]
    Tennisplayer2returny=Tennisplayer2returny[:-1]
    
    Tennisplayer1servis=float(Tennisplayer1servis)
    Tennisplayer2servis=float(Tennisplayer2servis)
    Tennisplayer1returny=float(Tennisplayer1returny)
    Tennisplayer2returny=float(Tennisplayer2returny)   
    
    B=Tennisplayer1ranking
    if B>200:
        B+=30
    if B>80 and B<201:
        B+=25
    if B>50 and B<81:
        B+=21 
    if B>30 and B<51:
        B+=17 
    if B>16 and B<31:
        B+=13
    if B>8 and B<17:
        B+=10
    if B>2 and B<9:
        B+=7        
    if B>0 and B<3:
        B+=4

    C=Tennisplayer2ranking
    if C>200:
        C+=30
    if C>80 and C<201:
        C+=25
    if C>50 and C<81:
        C+=21 
    if C>30 and C<51:
        C+=17 
    if C>16 and C<31:
        C+=13
    if C>8 and C<17:
        C+=10
    if C>2 and C<9:
        C+=7        
    if C>0 and C<3:
        C+=4
    P1=C/(B+C)
    P2=B/(B+C)
    diffskill=P1-P2
    diffskill=diffskill if diffskill>0 else -diffskill
    diff=0
    mini=min(P1,P2)
    diff=0.4-(((2*mini)**0.7)*0.4)
    
    avgserve=(Tennisplayer1servis+Tennisplayer2servis)/2
    avgreturn=(Tennisplayer1returny+Tennisplayer2returny)/2
    eqserve=avgserve/(avgserve+avgreturn)
    eqreturn=avgreturn/(avgserve+avgreturn)
    serpar=eqserve**0.7
    retpar=eqreturn**0.7
    PS1=0
    PS2=0
    PR1=0
    PR2=0
    K=0
    L=0
    if P1>P2:
        PS1=eqserve+(diff*(retpar/(retpar+serpar)))
        PS2=eqserve-(diff*(serpar/(retpar+serpar)))
        PR1=1-PS2
        PR2=1-PS1
    else:
        PS1=eqserve-(diff*(serpar/(retpar+serpar)))
        PS2=eqserve+(diff*(retpar/(retpar+serpar)))
        PR1=1-PS2
        PR2=1-PS1        
  
    if Surface =="Clay":
        K=PS1
        L=PS2  
        PS1+=-(K)*(L/10)
        PS2+=-(K)*(L/10)
        PR1=1-PS2
        PR2=1-PS1  
    if Surface =="Hard":
        pass
    if Surface=="Grass":
        K=PS1
        L=PS2                 
        PS1=K+((1-K)*(1/5))
        PS2=L+((1-L)*(1/5))
        PR1=1-PS2
        PR2=1-PS1
    else:
        pass
    
    Entry.insert(E4,0,PS1)
    print(PS1)
    Entry.insert(E5,0,PS2)
    print(PS2)
    
    
    
top = tkinter.Tk()
L1 = Label(top, text="Tennis calculator",).grid(row=0,column=1)
L2 = Label(top, text="Tennis player 1",).grid(row=1,column=0)
L3 = Label(top, text="Tennis player 2",).grid(row=2,column=0)
L4 = Label(top, text="Surface",).grid(row=3,column=0)
L5 = Label(top, text="P(Player 1 wins his serve)",).grid(row=4,column=0)
L6 = Label(top, text="P(Player 2 wins his serve)",).grid(row=5,column=0)
E1 = Entry(top, bd =5)
E1.grid(row=1,column=1)
E2 = Entry(top, bd =5)
E2.grid(row=2,column=1)
E3 = Entry(top, bd =5)
E3.grid(row=3,column=1)
E4 = Entry(top, bd =5)
E4.grid(row=4,column=1)
E5 = Entry(top, bd =5)
E5.grid(row=5,column=1)
B=Button(top, text ="Submit",command = proces).grid(row=6,column=1,)

top.mainloop()

0.747278495410766
0.8263511491799702
0.6305374138693921
0.7293782310808974
0.6840981192634574
0.7829389364749627
0.7067577886932703
0.7325738761765085


In [12]:
#tři vstupy, výstupy pravděpodobnost výhry zápasu s různým skóre
import tkinter
from tkinter import *
from tkinter import messagebox
import scipy.special
def proces():
    Tennisplayer1=Entry.get(E1)
    Tennisplayer2=Entry.get(E2)
    Surface=Entry.get(E3)
    h1=Tennisplayer(Tennisplayer1)
    h2=Tennisplayer(Tennisplayer2)
    Tennisplayer1ranking=h1.ranking(Tennisplayer1)
    Tennisplayer2ranking=h2.ranking(Tennisplayer2)
    Tennisplayer1servis=h1.servis(Tennisplayer1)
    if Tennisplayer1servis=="No data":
        Entry.insert(E4,0,"No data")
        print("No data")
        Entry.insert(E5,0,"No data")
        print("No data")
        return None
    Tennisplayer2servis=h2.servis(Tennisplayer2)
    if Tennisplayer2servis=="No data":
        Entry.insert(E4,0,"No data")
        print("No data")
        Entry.insert(E5,0,"No data")
        print("No data")
        return None       
    Tennisplayer1returny=h1.returny(Tennisplayer1)
    if Tennisplayer1returny=="No data":
        Entry.insert(E4,0,"No data")
        print("No data")
        Entry.insert(E5,0,"No data")
        print("No data")
        return None    
    Tennisplayer2returny=h2.returny(Tennisplayer2)
    if Tennisplayer2returny=="No data":
        Entry.insert(E4,0,"No data")
        print("No data")
        Entry.insert(E5,0,"No data")
        print("No data")
        return None 
    
    Tennisplayer1servis=Tennisplayer1servis[:-1]
    Tennisplayer2servis=Tennisplayer2servis[:-1]
    Tennisplayer1returny=Tennisplayer1returny[:-1]
    Tennisplayer2returny=Tennisplayer2returny[:-1]
    
    Tennisplayer1servis=float(Tennisplayer1servis)
    Tennisplayer2servis=float(Tennisplayer2servis)
    Tennisplayer1returny=float(Tennisplayer1returny)
    Tennisplayer2returny=float(Tennisplayer2returny)   
    
    B=Tennisplayer1ranking
    if B>200:
        B+=30
    if B>80 and B<201:
        B+=25
    if B>50 and B<81:
        B+=21 
    if B>30 and B<51:
        B+=17 
    if B>16 and B<31:
        B+=13
    if B>8 and B<17:
        B+=10
    if B>2 and B<9:
        B+=7        
    if B>0 and B<3:
        B+=4

    C=Tennisplayer2ranking
    if C>200:
        C+=30
    if C>80 and C<201:
        C+=25
    if C>50 and C<81:
        C+=21 
    if C>30 and C<51:
        C+=17 
    if C>16 and C<31:
        C+=13
    if C>8 and C<17:
        C+=10
    if C>2 and C<9:
        C+=7        
    if C>0 and C<3:
        C+=4
    P1=C/(B+C)
    P2=B/(B+C)
    diffskill=P1-P2
    diffskill=diffskill if diffskill>0 else -diffskill
    diff=0
    mini=min(P1,P2)
    diff=0.4-(((2*mini)**0.7)*0.4)
    
    avgserve=(Tennisplayer1servis+Tennisplayer2servis)/2
    avgreturn=(Tennisplayer1returny+Tennisplayer2returny)/2
    eqserve=avgserve/(avgserve+avgreturn)
    eqreturn=avgreturn/(avgserve+avgreturn)
    serpar=eqserve**0.7
    retpar=eqreturn**0.7
    PS1=0
    PS2=0
    PR1=0
    PR2=0
    K=0
    L=0
    if P1>P2:
        PS1=eqserve+(diff*(retpar/(retpar+serpar)))
        PS2=eqserve-(diff*(serpar/(retpar+serpar)))
        PR1=1-PS2
        PR2=1-PS1
    else:
        PS1=eqserve-(diff*(serpar/(retpar+serpar)))
        PS2=eqserve+(diff*(retpar/(retpar+serpar)))
        PR1=1-PS2
        PR2=1-PS1        
  
    if Surface =="Clay":
        K=PS1
        L=PS2  
        PS1+=-(K)*(L/10)
        PS2+=-(K)*(L/10)
        PR1=1-PS2
        PR2=1-PS1  
    if Surface =="Hard":
        pass
    if Surface=="Grass":
        K=PS1
        L=PS2                 
        PS1=K+((1-K)*(1/5))
        PS2=L+((1-L)*(1/5))
        PR1=1-PS2
        PR2=1-PS1
    else:
        pass
#z pravděpodobnosti servisu na pravděpodobnost setu
    G=0
    H=0
    J=0
    T1=0.5-((PS2-PS1)/2)
    T2=0.5+((PS2-PS1)/2)
    for k in range(0,6):
        serve1=(PS1**k)*((1-PS1)**((5-k)))*scipy.special.binom(5,k)
        for y in range(0,6):
            serve2=(PS2**y)*((1-PS2)**int((5-y)))*scipy.special.binom(5,y)
            if k+(5-y)>5:
                G+=serve1*serve2
            if k+(5-y)<5:
                H+=serve1*serve2
            if k+(5-y)==5:
                J+=serve1*serve2
            

    G+=J*PS1*((1-PS2))
    H+=J*PS2*((1-PS1))


    G+=(J*PS1*PS2*T1)+(J*(1-PS2)*(1-PS1)*T1)
    H+=(J*PS1*PS2*T2)+(J*(1-PS2)*(1-PS1)*T2)
   
    dvanula=G*G
    dvajedna=G*H*2*G
    jednadva=G*H*2*H
    nuladva=H*H
    
    Entry.insert(E4,0,dvanula)
    print(dvanula)
    Entry.insert(E5,0,dvajedna)
    print(dvajedna)
    Entry.insert(E6,0,jednadva)
    print(jednadva)
    Entry.insert(E7,0,nuladva)
    print(nuladva)
    
    
    
top = tkinter.Tk()
L1 = Label(top, text="Tennis calculator",).grid(row=0,column=1)
L2 = Label(top, text="Tennis player 1",).grid(row=1,column=0)
L3 = Label(top, text="Tennis player 2",).grid(row=2,column=0)
L4 = Label(top, text="Surface",).grid(row=3,column=0)
L5 = Label(top, text="P(Player 1 wins 2:0)",).grid(row=4,column=0)
L6 = Label(top, text="P(Player 1 wins 2:1)",).grid(row=5,column=0)
L7 = Label(top, text="P(Player 2 wins 2:0)",).grid(row=6,column=0)
L8 = Label(top, text="P(Player 2 wins 2:1)",).grid(row=7,column=0)
E1 = Entry(top, bd =5)
E1.grid(row=1,column=1)
E2 = Entry(top, bd =5)
E2.grid(row=2,column=1)
E3 = Entry(top, bd =5)
E3.grid(row=3,column=1)
E4 = Entry(top, bd =5)
E4.grid(row=4,column=1)
E5 = Entry(top, bd =5)
E5.grid(row=5,column=1)
E6 = Entry(top, bd =5)
E6.grid(row=6,column=1)
E7 = Entry(top, bd =5)
E7.grid(row=7,column=1)
B=Button(top, text ="Submit",command = proces).grid(row=8,column=1,)

top.mainloop()

0.14801135664641962
0.18213613242897828
0.29128603637545053
0.3785664745491519
